In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
%cd /content/drive/MyDrive/GitHub/Detecting-Generated-Abstract/

/content/drive/MyDrive/GitHub/Detecting-Generated-Abstract


In [ ]:
%pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.5 MB/s eta 0:00:00


In [ ]:
!ls

In [ ]:
!python --version

In [ ]:
import torch

In [ ]:
!python preprocess.py


In [ ]:
!python train.py


2023-03-06 02:53:46.195299: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-06 02:53:47.160920: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-06 02:53:47.161029: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-06 02:53:47.161047: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

In [ ]:
!python test.py

2023-03-06 07:31:51.261451: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-06 07:31:52.131087: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-06 07:31:52.131186: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-06 07:31:52.131205: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

In [ ]:
!ls

drive  sample_data


In [ ]:
# GPT2 Like Model
import torch
from torch.optim import Adam
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config

# Initialize the GPT model and tokenizer
gpt_config = GPT2Config.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = GPT2LMHeadModel(gpt_config).cuda()

# Initialize the detector model and tokenizer
detector_tokenizer, detector_model = load_model("best_model.pt")

# Set up the optimizers
gpt_optimizer = Adam(gpt_model.parameters(), lr=2e-5)
detector_optimizer = Adam(detector_model.parameters(), lr=2e-5)

# Adversarial training loop
num_epochs = 5
for epoch in range(num_epochs):
    for batch in dataset:

        # Update the GPT model to generate text that fools the detector
        gpt_optimizer.zero_grad()
        inputs = tokenizer(batch, return_tensors="pt").to("cuda")
        outputs = gpt_model(**inputs)
        generated_text = tokenizer.decode(outputs.logits.argmax(dim=-1).cpu().numpy())
        input_ids, attention_mask = preprocess_text(detector_tokenizer, generated_text, max_length=512)
        predicted_label = get_prediction(detector_model, input_ids, attention_mask)
        adversarial_loss = -torch.log(1 - predicted_label)
        adversarial_loss.backward()
        gpt_optimizer.step()

        # Update the detector model using the same batch
        detector_optimizer.zero_grad()
        input_ids, attention_mask = preprocess_text(detector_tokenizer, batch, max_length=512)
        ground_truth_labels = torch.tensor(batch_labels).cuda()
        outputs = detector_model(input_ids, attention_mask=attention_mask)
        predicted_labels = torch.argmax(outputs.logits, dim=-1)
        detector_loss = torch.nn.CrossEntropyLoss()(predicted_labels, ground_truth_labels)
        detector_loss.backward()
        detector_optimizer.step()

# Save the fine-tuned GPT model
gpt_model.save_pretrained("fine_tuned_gpt")


# AutoGPT on Colab

In [ ]:
%cd /content
!git clone https://github.com/Significant-Gravitas/Auto-GPT.git

In [ ]:
%python -m pip install -r requirements.txt
%python -m autogpt